<div style= "text-align: right">
    <p style= "text-align: right; font-weight: bold; font-size: x-large;">FIT3182 Big Data Management and Processing</p>
    <p style= "text-align: right; font-weight: bold; font-size: large;">Assignment 2</p>
    <p style= "text-align: right">Foo Kai Yan</p>
    <p style= "text-align: right">kfoo0012@student.monash.edu<br><br><i>33085625<br><br><i>5<sup>th</sup> May 2024</i></p>
<div>
<hr style="border-color: black;">

## Student Statement
The assignment was completed with the assistance of some code obtained from seminar/tutorial/lab/applied class.

### Installing PyMongo

In [1]:
!pip install pymongo
!pip install pygeohash

### Import required Libraries

In [2]:
import os
import json
import time
import datetime
import pygeohash as pgh
from pprint import pprint
from pymongo import MongoClient
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf # spark
from pyspark.streaming import StreamingContext # spark streaming
from pyspark.sql.functions import col, split, element_at, when

### Check working directory

In [3]:
os.getcwd()

'/home/student/kfoo0012'

### Set os environment

In [4]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.3.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0 pyspark-shell'

### Set host ip

In [5]:
# hostip obtained using `ipconfig` command in command prompt
hostip = "192.168.68.58"

### Streaming Application Processing Data

In [6]:
topic = "Climate, Hotspot_AQUA, Hotspot_TERRA"

spark = (
    SparkSession.builder
    .master('local[*]') # local[*] MEANS you are using all the available processors
    .appName('Streaming Climate Data')
    .getOrCreate()
)

In [7]:
# This is basically connecting to kafka server
topic_stream_df = (
    spark.readStream.format('kafka') # 'kafka' because u are receiving from kafka
    .option('kafka.bootstrap.servers', f'{hostip}:9092')
    .option('subscribe', topic)
    .load()
)

In [8]:
data_sdf = topic_stream_df.select('value')

In [9]:
def addtoDatabase_OLD(batch_df, batch_id):
    all_data = batch_df.collect()  # Returns all elements as an array
    # Send stream data to be transformed & analysed
    producer_data = [row.asDict() for row in all_data]
#     new_data = process_producer_data(data)
    for eachdata in range(len(producer_data)): 
        temp_var = producer_data[eachdata].get('value')
        # print("eachdata: ", eachdata)
        # Convert byte array to string
        newdata = temp_var.decode('utf-8')
        # Convert string to JSON
        produced_data = json.loads(newdata)
        producer_data[eachdata] = produced_data
        print("produced_data: ", producer_data)

def addtoDatabase_old(batch_df, batch_id):
    all_data = batch_df.collect()  # Returns all elements as an array
    # Send stream data to be transformed & analysed
    producer_data = [row.asDict() for row in all_data]

    for eachdata in range(len(producer_data)):
        # Check if 'value' key exists and is a bytearray
        if 'value' in producer_data[eachdata] and isinstance(producer_data[eachdata]['value'], bytearray):
            # Convert byte array to string
            newdata = producer_data[eachdata]['value'].decode('utf-8')
            # Convert string to JSON
            produced_data = json.loads(newdata)
            producer_data[eachdata] = produced_data
        # If 'value' is not a bytearray, keep the original dictionary

    print("produced_data: ", producer_data)
    
    for producer in producer_data:
        producer_id = producer["producer_id"]

        climate_record = {}
        aqua_hotspots_record = []
        terra_hotspots_record = []

        # Sort data depending on the producer_id information
        if producer_id == "producer1_climate":
            climate_record = producer
            print("climate record", climate_record)
        elif producer_id == "producer2_hotspot_aqua":
            aqua_hotspots_record.append(producer)
            print("aqua_hotspots_record", aqua_hotspots_record)
        elif producer_id == "producer3_hotspot_terra":
            terra_hotspots_record.append(producer)
            print("terra_hotspots_record", terra_hotspots_record)
            
        hotspots = []
        
        
            
        # Get Climate longitude and latitude
        climate_latitude = climate_record.get("latitude")
        climate_longitude = climate_record.get("longitude") 

#         print("climate_latitude: ", climate_latitude)
#         print("climate_longitude: ", climate_longitude)
        
        # Process aqua_hotspots_record
        for each_record in aqua_hotspots_record:
            #if geohash_proximity(each_record, climate_record):
            if (pgh.encode(each_record["longitude"], each_record["latitude"], precision = 3) == pgh.encode(climate_record.get("longitude"), climate_record.get("latitude"), precision = 3)):
                #each_record["date"] = climate_record.get("date")
                hotspots.append(each_record)
                print("hotspot", hotspots)

def addtoDatabase(batch_df, batch_id): # NO ERROR OUH YEAAA
    all_data = batch_df.collect()  # Returns all elements as an array
    producer_data = [row.asDict() for row in all_data]

    climate_record = [] #{}
    aqua_hotspots_record = []
    terra_hotspots_record = []
    hotspots = []

    for eachdata in range(len(producer_data)):
        # Check if 'value' key exists and is a bytearray
        if 'value' in producer_data[eachdata] and isinstance(producer_data[eachdata]['value'], bytearray):
            # Convert byte array to string and then to JSON
            newdata = producer_data[eachdata]['value'].decode('utf-8')
            produced_data = json.loads(newdata)
            producer_data[eachdata] = produced_data

    for producer in producer_data:
        producer_id = producer["producer_id"]

#         climate_record = {}
#         aqua_hotspots_record = []
#         terra_hotspots_record = []

        # Sort data depending on the producer_id information
        if producer_id == "producer1_climate":
            climate_record.append(producer)
            # print("climate record", climate_record)
        elif producer_id == "producer2_hotspot_aqua":
            aqua_hotspots_record.append(producer)
            # print("aqua_hotspots_record", aqua_hotspots_record)
        elif producer_id == "producer3_hotspot_terra":
            terra_hotspots_record.append(producer)
            # print("terra_hotspots_record", terra_hotspots_record)

    # If there are no climate records, skip processing this batch
    # Check if the dictionary is empty
    if len(climate_record) == 1:
        climate_record = climate_record
    elif len(climate_record) > 1:
        climate_record = climate_record[0]
    else:
        # print("No climate record present in this batch. Skipping...")
        return  # Exit the function

    # If there is a climate record, continue processing
    # ... rest of your code to process the records ...

#     print("Climate records:", climate_record)
#     print("Aqua hotspots records:", aqua_hotspots_record)
#     print("Terra hotspots records:", terra_hotspots_record)
    # Further processing can be done here, such as combining data or storing in a database

    # Process terra_hotspots_record
    print("out loop")
    for each_record in terra_hotspots_record:
        print("in loop terra")
        if geohash_proximity(each_record, climate_record):
            #each_record["date"] = climate_record.get("date")
            hotspots.append(each_record)
            print("hotspot", hotspots)
            
    for each_record in aqua_hotspots_record:
        print("in loop aqua")
        if geohash_proximity(each_record, climate_record):
            #each_record["date"] = climate_record.get("date")
            hotspots.append(each_record)
            print("hotspot", hotspots)
            
    # Analyse hotspots data, find if any are close by & merge
    new_hotspots_record = hotspots
    # Merge hotspots with climate depending if close & label if natural or other
    new_climate_record = process_climate(climate_record, new_hotspots_record)
            
    return hotspots

# Check proximity based on geohash, precision 3
def geohash_proximity(record, climate):
    record_geohash = pgh.encode(record["longitude"], record["latitude"], precision = 3)
    climate_geohash = pgh.encode(climate[0]["longitude"], climate[0]["latitude"], precision = 3)
    return record_geohash == climate_geohash

    # Only save data with contents
#     if len(new_data) > 1:

#         client = MongoClient(f'mongodb://{hostip}:27017/')

#         db = client.fit3182_assignment_db
#         climate_collection = db.climate_collection

#         # Insert climate data into database
#         climate_collection.insert_one(new_data)
#         pprint(new_data)

#         client.close()

In [10]:
db_writer = (
    data_sdf
    .writeStream
    .outputMode('append')
    # collect data for 10 seconds
    .trigger(processingTime='10 seconds')
    # each batch of 10 seconds will run func
    .foreachBatch(addtoDatabase))

In [11]:
try:
    query = db_writer.start()
    query.awaitTermination()
except KeyboardInterrupt:
    print('Interrupted by CTRL-C. Stopping query.')
finally:
    query.stop()

out loop
in loop terra
in loop terra
in loop terra
hotspot [{'latitude': -37.9029, 'longitude': 143.5946, 'confidence': 73.0, 'surface_temperature_celcius': 47.0, 'created_time': '10:14:53', 'producer_id': 'producer3_hotspot_terra'}]
in loop terra
hotspot [{'latitude': -37.9029, 'longitude': 143.5946, 'confidence': 73.0, 'surface_temperature_celcius': 47.0, 'created_time': '10:14:53', 'producer_id': 'producer3_hotspot_terra'}, {'latitude': -37.636, 'longitude': 149.33, 'confidence': 94.0, 'surface_temperature_celcius': 43.0, 'created_time': '10:14:53', 'producer_id': 'producer3_hotspot_terra'}]
in loop aqua
hotspot [{'latitude': -37.9029, 'longitude': 143.5946, 'confidence': 73.0, 'surface_temperature_celcius': 47.0, 'created_time': '10:14:53', 'producer_id': 'producer3_hotspot_terra'}, {'latitude': -37.636, 'longitude': 149.33, 'confidence': 94.0, 'surface_temperature_celcius': 43.0, 'created_time': '10:14:53', 'producer_id': 'producer3_hotspot_terra'}, {'latitude': -37.9009, 'longitu

StreamingQueryException: Query [id = c929529b-12fb-45f9-8a4c-79eb58a9fb17, runId = 0aec3f9c-4149-4ba6-b953-a9bf26e35987] terminated with exception: An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/py4j/clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
  File "/opt/conda/lib/python3.8/site-packages/pyspark/sql/utils.py", line 272, in call
    raise e
  File "/opt/conda/lib/python3.8/site-packages/pyspark/sql/utils.py", line 269, in call
    self.func(DataFrame(jdf, self.session), batch_id)
  File "/tmp/ipykernel_1307/3193697816.py", line 142, in addtoDatabase
    new_climate_record = process_climate(climate_record, new_hotspots_record)
NameError: name 'process_climate' is not defined


client = MongoClient(f'mongodb://{hostip}:27017/')
db = client.fit3182_assignment_db
cursor = db.climate_collection.find({})

for document in cursor:
    pprint(document)

In [ ]:
# Check proximity based on geohash, precision 5
def geohash_proximity_5(record, climate):
    record_geohash = pgh.encode(record.get("longitude"), record.get("latitude"), precision = 5)
    climate_geohash = pgh.encode(climate.get("longitude"), climate.get("latitude"), precision = 5)
    return record_geohash == climate_geohash

# Check proximity based on geohash, precision 3
def geohash_proximity(record, climate):
    record_geohash = pgh.encode(record.get("longitude"), record.get("latitude"), precision = 3)
    climate_geohash = pgh.encode(climate.get("longitude"), climate.get("latitude"), precision = 3)
    return record_geohash == climate_geohash

def process_producer_data(producer_data):
    print("producer_data: ", producer_data)
    # Initialize empty lists for aqua_hotspots_record, terra_hotspots_record, and an empty dictionary for climate_record
    climate_record = {}
    aqua_hotspots_record = []
    terra_hotspots_record = []
    
    # Producer 1: Climate topic_name = "Climate", ["producer_id"] = "producer1_climate"
    # Producer 2: AQUA topic_name = "Hotspot_AQUA", ["producer_id"] = "producer2_hotspot_aqua"
    # Producer 3: TERRA topic_name = "Hotspot_TERRA", ["producer_id"] = "producer3_hotspot_terra"
    
    # For each item from the data batch from the Kafka stream 
    for eachdata in range(len(producer_data)): 
        temp_var = producer_data[eachdata].get("value")
        # print("eachdata: ", eachdata)
        # Convert byte array to string
        newdata = temp_var.decode('utf-8')
        # Convert string to JSON
        produced_data = json.loads(newdata)
        print("produced_data: ", produced_data)
        producer_id = produced_data["producer_id"]

        # Sort data depending on the producer_id information
        if producer_id == "producer1_climate":
            climate_record = produced_data
        elif producer_id == "producer2_hotspot_aqua":
            aqua_hotspots_record.append(produced_data)
        elif producer_id == "producer3_hotspot_terra":
            terra_hotspots_record.append(produced_data)

    # Analyse hotspots data, find if any are close by & merge
    new_hotspots_record = process_hotspots(aqua_hotspots_record, terra_hotspots_record, climate_record)
    # Merge hotspots with climate depending if close & label if natural or other
    new_climate_record = process_climate(climate_record, new_hotspots_record)

    return new_climate_record

def process_hotspots(aqua_hotspots_record, terra_hotspots_record, climate_record):
    # Initialize empty array for hotspots records from both terra and aqua
    hotspots = []
    
    print("climate_record: ", climate_record)
    
    # Get Climate longitude and latitude
    climate_latitude = climate_record.get("latitude")
    climate_longitude = climate_record.get("longitude") 
    
    print("climate_latitude: ", climate_latitude)
    print("climate_longitude: ", climate_longitude)
    
    # Process aqua_hotspots_record
    for each_record in aqua_hotspots_record:
        if geohash_proximity(each_record, climate_record):
            each_record["date"] = climate_record.get("date")
            hotspots.append(each_record)
            
    # Process terra_hotspots_record
    for each_record in terra_hotspots_record:
        if geohash_proximity(each_record, climate_record):
            each_record["date"] = climate_record.get("date")
            hotspots.append(each_record)
            
    return hotspots

def process_climate(climate_record, hotspots_record):
    # Initialize empty array for fire events records
    fire_events = []
    for each_record in hotspots_record:
        fire_event = process_fire_event(climate_record, each_record)
        if fire_event is not None:
            fire_events.append(fire_event)
    climate_record["fire_events"] = fire_events
    return climate_record

def process_fire_event(climate_record, hotspots_record):
    # Initialize an empty dictionary to store fire event data
    fire_happening = {}

    # Check if the hotspot is geographically close to the climate data with a precision of 5
    if geohash_proximity_5(hotspots_record, climate_record) is True:
        
        print("hotspots_record: ", hotspots_record)
        
        # Calculate the average surface temperature from both hotspot and climate data
        average_temp = (hotspots_record.get("surface_temperature_celcius") + hotspots_record.get("surface_temperature_celcius")) / 2
        fire_happening["average_surface_temp"] = average_temp

        # Calculate the average confidence level from both hotspot and climate data
        average_confidence = (hotspots_record.get("confidence") + hotspots_record.get("confidence")) / 2
        fire_happening["confidence"] = average_confidence

        # Determine the cause of the fire based on climate conditions
        air_temp = climate_record.get("air_temperature_celcius")
        solar_irradiance = climate_record.get("GHI_w/m2")
        if air_temp > 20 and solar_irradiance > 180:
            fire_happening["cause"] = "natural"
        else:
            fire_happening["cause"] = "others"
        
        print("fire_happening: ", fire_happening)

        # Return the dictionary containing fire event data
        return fire_happening
    # If the hotspot is not close to the climate data, return None
    return None

